In [521]:
import requests
import json
import pandas as pd
from datetime import datetime

### Set the time for Now

In [522]:
now = datetime.utcnow()

### Get bearer token

In [523]:
username = "timothy_woodland@outlook.com"
password = "@j-tLtrR8F"
client_secret = 'wZaRN7rpjn3FoNyF5IFuxg9uMzYJcvOoQ8QWiIqS3hfk6gLhVlG57j5YNoZL2Rtc'

In [524]:
payload = {'client_id':'tado-web-app',
            'grant_type':'password',
            'scope':'home.user',
            'username':username,
            'password':password,
            'client_secret':client_secret}

In [525]:
token_r = requests.post('https://auth.tado.com/oauth/token', params=payload)
token = token_r.json()["access_token"]
# print(token)

### Set Header for future Requests

In [526]:
header = {'Authorization' : "Bearer " + token}

### Get Home ID

In [527]:
home_id_r = requests.get("https://my.tado.com/api/v1/me", headers = header)
home_id = home_id_r.json()['homeId']
# print(home_id)

### Get name of home

In [528]:
home_url = "https://my.tado.com/api/v2/homes/"+str(home_id)
home_r = requests.get(home_url , headers = header)

In [529]:
home = home_r.json()["name"]

In [530]:
print(home)

Leslie Road


### Get zone data

In [531]:
zones_list_url = home_url+"/zones"

In [532]:
zone_list_r = requests.get(zones_list_url , headers = header)

In [533]:
zone_list_json = zone_list_r.json()
# print(json.dumps(zone_list_json, indent=1))

In [534]:
zone_list = []
zone_dict = {}
counter = 1


for zone_data in zone_list_json:
    zone_id = zone_data["id"]
    zone_name = zone_data["name"]

    zone_list.append(zone_id)
    
    zone_dict[zone_id] = {"time":str(now), "zone":zone_name}

    counter = counter + 1

### Get data from each zone and put into a dataframe

In [536]:
for zone in zone_list:
    zone_url = zones_list_url+"/"+str(zone)+"/state"
    zone_r = requests.get(zone_url, headers = header)
    zone_json = zone_r.json()

    mode = zone_json["tadoMode"]
    geo_override = zone_json["geolocationOverride"]
    type = zone_json["setting"]["type"]
    status = zone_json["setting"]["power"]
    set_temp = zone_json["setting"]["temperature"]["celsius"]
    power_level = zone_json["activityDataPoints"]["heatingPower"]["percentage"]
    zone_temp = zone_json["sensorDataPoints"]["insideTemperature"]["celsius"]
    zone_humidity = zone_json["sensorDataPoints"]["humidity"]["percentage"]

    zone_data_dict = {"mode":mode, "geo_override":geo_override, "type":type, "status":status, "set_temp":set_temp, "power_level":power_level, "zone_temp":zone_temp, "zone_humidity":zone_humidity }

    zone_dict[zone].update(zone_data_dict)
    

In [538]:
zone_df = pd.DataFrame.from_dict(zone_dict, orient="index")

                         time         zone  mode  geo_override     type  \
2  2023-12-16 13:41:49.290898        Study  HOME         False  HEATING   
6  2023-12-16 13:41:49.290898      Kitchen  HOME         False  HEATING   
3  2023-12-16 13:41:49.290898      Bedroom  HOME         False  HEATING   
1  2023-12-16 13:41:49.290898  Living Room  HOME         False  HEATING   

  status  set_temp  power_level  zone_temp  zone_humidity  
2     ON      20.0        100.0      19.44           66.1  
6     ON      21.0          0.0      21.83           57.4  
3     ON      20.0         12.0      20.04           62.6  
1     ON      20.0         21.0      19.79           63.4  


In [539]:
zone_df = zone_df.reset_index()
zone_df= zone_df.rename(columns={"index":"zone_id"})

   zone_id                        time         zone  mode  geo_override  \
0        2  2023-12-16 13:41:49.290898        Study  HOME         False   
1        6  2023-12-16 13:41:49.290898      Kitchen  HOME         False   
2        3  2023-12-16 13:41:49.290898      Bedroom  HOME         False   
3        1  2023-12-16 13:41:49.290898  Living Room  HOME         False   

      type status  set_temp  power_level  zone_temp  zone_humidity  
0  HEATING     ON      20.0        100.0      19.44           66.1  
1  HEATING     ON      21.0          0.0      21.83           57.4  
2  HEATING     ON      20.0         12.0      20.04           62.6  
3  HEATING     ON      20.0         21.0      19.79           63.4  


In [540]:
zone_df = zone_df.set_index("time")

                            zone_id         zone  mode  geo_override     type  \
time                                                                            
2023-12-16 13:41:49.290898        2        Study  HOME         False  HEATING   
2023-12-16 13:41:49.290898        6      Kitchen  HOME         False  HEATING   
2023-12-16 13:41:49.290898        3      Bedroom  HOME         False  HEATING   
2023-12-16 13:41:49.290898        1  Living Room  HOME         False  HEATING   

                           status  set_temp  power_level  zone_temp  \
time                                                                  
2023-12-16 13:41:49.290898     ON      20.0        100.0      19.44   
2023-12-16 13:41:49.290898     ON      21.0          0.0      21.83   
2023-12-16 13:41:49.290898     ON      20.0         12.0      20.04   
2023-12-16 13:41:49.290898     ON      20.0         21.0      19.79   

                            zone_humidity  
time                                      

### Testing Get data from a Zone

In [ ]:
zone = 1

In [ ]:
zone_url = zones_list_url+"/"+str(zone)+"/state"

In [ ]:
zone_r = requests.get(zone_url , headers = header)

In [ ]:
zone_json = zone_r.json()

In [ ]:
print(zone_json)

In [ ]:
mode = zone_json["tadoMode"]
geo_override = zone_json["geolocationOverride"]
type = zone_json["setting"]["type"]
status = zone_json["setting"]["power"]
set_temp = zone_json["setting"]["temperature"]["celsius"]
power_level = zone_json["activityDataPoints"]["heatingPower"]["percentage"]
zone_temp = zone_json["sensorDataPoints"]["insideTemperature"]["celsius"]
zone_temp = zone_json["sensorDataPoints"]["humidity"]["percentage"]

In [ ]:
zone_temp = zone_json["sensorDataPoints"]["humidity"]["percentage"]

In [ ]:
print(json.dumps(zone_r.json(), indent=2))

In [ ]:
json_test = zone_r.json()

In [ ]:
json_test["sensorDataPoints"]["insideTemperature"]["celsius"]

In [ ]:
json_test["sensorDataPoints"]["humidity"]["percentage"]

### Test Weather

In [ ]:
weather_url = "https://my.tado.com/api/v2/homes/"+str(home_id)+"/weather"

In [ ]:
r = requests.get(weather_url , headers = header)

In [ ]:
# print(json.dumps(r.json(), indent=2))

In [ ]:
test_json = r.json()

In [ ]:
json_df = pd.DataFrame.from_dict(test_json)

In [ ]:
test_json["weatherState"]["value"]

In [ ]:
test_json["outsideTemperature"]["celsius"]

### Get Weather data and put into a dataframe

In [ ]:
weather_json = r.json()

In [ ]:
weather_data = [weather_json["outsideTemperature"]["celsius"], weather_json["weatherState"]["value"], "tado"]

In [ ]:
weather_columns = ["temperature_C", "state", "source"]

In [ ]:
weather_df = pd.DataFrame(data=[weather_data], index=[now], columns=weather_columns)